In [1]:
import pymysql
import pandas as pd

import pandas as pd
import numpy as np
from difflib import SequenceMatcher

import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import glob, tqdm, warnings

In [13]:
connection = pymysql.connect(host='localhost', user='root', passwd='password',db='resursenew') 
cursor = connection.cursor()

pd.read_sql("SHOW DATABASES;", connection)

,Database
0,duplicates
1,information_schema
2,mysql
3,performance_schema
4,resurse_crestine
5,resursenew
6,sys


In [14]:

query_ref = """SELECT * FROM resursenew.resources limit 5;"""
display(pd.read_sql(query_ref, connection))


,id,user_id,res_type_id,res_category_id,language_id,alb_vol_id,author_id,age_category_id,stats_id,title,visible,image_link,rating_votes,rating_sum,rating_avg,res_length,created_at,updated_at,comments_blocked,restricted
0,1,1,1,5,1,4464,6265,1,1,Întrucât atârnă de voi,1,None,1,10,10.0,3,2006-01-18,2014-09-15 20:00:54,0,0
1,2,1,1,5,2,4575,173,1,2,God will make a way,1,None,0,0,0.0,3,2006-01-18,2011-12-12 16:45:23,0,0
2,3,1,1,5,2,139,240,1,3,King of Glory,1,None,0,0,0.0,3,2006-01-18,2011-12-12 16:56:32,0,0
3,4,1,1,5,1,4210,78,1,4,Curgi ca un râu de slavă,1,None,0,0,0.0,1,2006-01-18,2011-12-12 17:05:55,0,0
4,5,1,1,5,1,1,7999,1,5,Inima mea tânjește,1,None,3,30,10.0,2,2006-01-18,2018-03-09 17:15:50,0,0


In [15]:

query_ref = """SELECT * FROM resursenew.res_type limit 5;"""
display(pd.read_sql(query_ref, connection))


,id,res_lengths,name,name_singular,slug,domain,rtypes_group
0,1,667;1333,Acorduri,acorduri,acorduri,text,1
1,2,477;953,Cântece,cântec,cantece,text,1
2,3,2246;4492,Devoționale,devoțional,devotionale,text,5
3,4,4309;8619,Editoriale,editorial,editoriale,text,5
4,5,5314;10628,Eseuri,eseu,eseuri,text,5


In [16]:

query_ref = """SELECT * FROM resursenew.text_details limit 5;"""
display(pd.read_sql(query_ref, connection))


,id,resources_id,content,description,link,status_gramatical,status_diacritice,status_autentic,content_length
0,1,1,VERSE 1:\r\nb\t\t F#\r\nÎntrucât atârnă de ...,None,None,5,7,9,0
1,2,2,D\r\nGod will make a way\r\n ...,None,None,5,7,9,0
2,3,3,E B\r\nWho is this King o...,None,None,5,7,9,0
3,4,4,E\r\n Curgi ca un râu de slavă\r\n ...,None,None,5,7,9,0
4,5,5,C F C G\r\nInima mea...,None,None,5,7,10,0


In [27]:
resource_type2remove =  [15,17,18,20,22,30,31,29,40,48,35,12]
interval = 5600

query_ref = """
  SELECT
    resources.id,
    resources.res_type_id,
    resources.language_id,
    resources.title,
    resources.created_at,
    text_details.content  
  FROM
    resources
  LEFT JOIN text_details ON resources.id = text_details.resources_id
  WHERE
    text_details.content is not null and resources.created_at < (NOW() - INTERVAL """ + str(interval) + " HOUR) and  resources.res_type_id not in (""" + ', '.join([str(el) for el in resource_type2remove]) + ") "
  
df_ref = pd.read_sql(query_ref, connection)

df_ref.head()

,id,res_type_id,language_id,title,created_at,content
0,1,1,1,Întrucât atârnă de voi,2006-01-18 00:00:00,VERSE 1:\r\nb\t\t F#\r\nÎntrucât atârnă de ...
1,2,1,2,God will make a way,2006-01-18 00:00:00,D\r\nGod will make a way\r\n ...
2,3,1,2,King of Glory,2006-01-18 00:00:00,E B\r\nWho is this King o...
3,4,1,1,Curgi ca un râu de slavă,2006-01-18 00:00:00,E\r\n Curgi ca un râu de slavă\r\n ...
4,5,1,1,Inima mea tânjește,2006-01-18 00:00:00,C F C G\r\nInima mea...


In [28]:
df_ref.res_type_id.unique()

array([ 1,  2,  7,  3, 10, 43,  4,  5,  9,  6, 46,  8, 39, 11, 42, 45, 37,
       47])

'15, 17, 18, 20, 22, 30, 31, 29, 40, 48, 35, 12'